In [1]:
#!conda install -c conda-forge beautifulsoup4 --yes 

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0 conda-forge

beautifulsoup4 100% |################################| Time: 0:00:00   8.30 MB/s


# Packages import

In [1]:
import urllib.request as url
from bs4 import BeautifulSoup as bs
import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import pandas as pd
import numpy as np
import re

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Web scraping

In [2]:
w_url = 'https://fr.wikipedia.org/wiki/Arrondissements_de_Paris'
w_page = requests.get(w_url).text
soup = bs(w_page,'lxml')

In [40]:
w_class='wikitable sortable alternance'
rows = []
for (index,w_row) in enumerate(soup.find('table',{'class':w_class}).find('tbody').find_all('tr')):
    row = [x.text.replace('\n','') for x in w_row.find_all('td')]
    if (len(row)==15 or len(row)==16) and index<23:
        #print(len(row),row)
        rows.append([
            75000+int(re.findall(r'\d+', row[0])[0]),
            row[1].replace('[note 1]','').split(' ')[0].split(',')[0],
            int(row[9].replace('\xa0',''))])

df_postcode = pd.DataFrame(rows,columns = ['District','Name','Population'])
print(df_postcode.shape)
df_postcode.head(20)

(20, 3)


,District,Name,Population
0,75001,Louvre,16252
1,75002,Bourse,20260
2,75003,Temple,34788
3,75004,Hôtel-de-Ville,27487
4,75005,Panthéon,59108
5,75006,Luxembourg,40916
6,75007,Palais-Bourbon,52512
7,75008,Élysée,36453
8,75009,Opéra,59629
9,75010,Entrepôt,91932


# Acquiring geographical data 
### from data.gouv.fr / nodonnees

In [4]:
!rm 'Geospatial_data.csv.gz'
!rm 'Geospatial_data.csv'

!wget -q -O 'Geospatial_data.csv.gz' http://www.nosdonnees.fr/wiki/images/b/b5/EUCircos_Regions_departements_circonscriptions_communes_gps.csv.gz

!gzip -d 'Geospatial_data.csv.gz'

rm: cannot remove ‘Geospatial_data.csv.gz’: No such file or directory


In [5]:
df_geo = pd.read_csv('Geospatial_data.csv',sep =';')
df_geo[df_geo['nom_commune']=='Paris'].head(5)

,EU_circo,code_région,nom_région,chef-lieu_région,numéro_département,nom_département,préfecture,numéro_circonscription,nom_commune,codes_postaux,code_insee,latitude,longitude,éloignement
30893,Île-de-France,11,Île-de-France,Paris,75,Paris,Paris,1,Paris,75001 75002 75003 75004,75056,NaN,NaN,NaN
30894,Île-de-France,11,Île-de-France,Paris,75,Paris,Paris,2,Paris,75005 75006,75056,NaN,NaN,NaN
30895,Île-de-France,11,Île-de-France,Paris,75,Paris,Paris,3,Paris,75006 75007,75056,NaN,NaN,NaN
30896,Île-de-France,11,Île-de-France,Paris,75,Paris,Paris,4,Paris,75008 75009,75056,NaN,NaN,NaN
30897,Île-de-France,11,Île-de-France,Paris,75,Paris,Paris,5,Paris,75010,75056,NaN,NaN,NaN


## Fail !
### Try with Google

In [6]:
#!conda install -c conda-forge googlemaps --yes 

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
googlemaps                2.5.1                      py_0    conda-forge


In [7]:
import googlemaps
#https://github.com/googlemaps/google-maps-services-python

In [57]:
# The code was removed by Watson Studio for sharing.

In [11]:
geocode_result = gmaps.geocode('75015, France')
geocode_result

[{'address_components': [{'long_name': '75015',
    'short_name': '75015',
    'types': ['postal_code']},
   {'long_name': 'Paris',
    'short_name': 'Paris',
    'types': ['locality', 'political']},
   {'long_name': 'Paris',
    'short_name': 'Paris',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Île-de-France',
    'short_name': 'Île-de-France',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'France',
    'short_name': 'FR',
    'types': ['country', 'political']}],
  'formatted_address': '75015 Paris, France',
  'geometry': {'bounds': {'northeast': {'lat': 48.87359000000001,
     'lng': 2.3519189},
    'southwest': {'lat': 48.8252116, 'lng': 2.2629819}},
   'location': {'lat': 48.8421616, 'lng': 2.2927665},
   'location_type': 'APPROXIMATE',
   'viewport': {'northeast': {'lat': 48.87359000000001, 'lng': 2.3519189},
    'southwest': {'lat': 48.8252116, 'lng': 2.2629819}}},
  'place_id': 'ChIJ9WBuGRRw5kcR4FHY4caCCxw',
  'typ

In [12]:
geocode_result[0]['geometry']['location']

{'lat': 48.8421616, 'lng': 2.2927665}

In [22]:
locations=[]
for index,row in df_postcode.iterrows():
    address = '{}, France'.format(row['District'])
    print(address)
    geocode_result = gmaps.geocode(address)
    locations.append([
        row['District'],
        geocode_result[0]['geometry']['location']['lat'],
        geocode_result[0]['geometry']['location']['lng']
    ])

locations

75001, France
75002, France
75003, France
75004, France
75005, France
75006, France
75007, France
75008, France
75009, France
75010, France
75011, France
75012, France
75013, France
75014, France
75015, France
75016, France
75017, France
75018, France
75019, France
75020, France


[[48.8640493, 2.3310526],
 [48.8675641, 2.34399],
 [48.8634799, 2.3591145],
 [48.8534275, 2.3582788],
 [48.8434912, 2.3518339],
 [48.8488576, 2.3354223],
 [48.85433450000001, 2.3134029],
 [48.8718722, 2.3176432],
 [48.8790183, 2.3379063],
 [48.8785618, 2.3603689],
 [48.85799300000001, 2.381153],
 [48.8293647, 2.4265406],
 [48.830759, 2.359204],
 [48.8314408, 2.3255684],
 [48.8421616, 2.2927665],
 [48.8530933, 2.2487626],
 [48.891986, 2.319287],
 [48.891305, 2.3529867],
 [48.89061359999999, 2.3867083],
 [48.8599825, 2.4066412]]

In [36]:
df_geo=pd.DataFrame(data=locations,columns=['District','lat','lng'])


In [53]:
df_paris = df_postcode.set_index('District').join(df_geo.set_index('District')).reset_index()
df_paris

,District,Name,Population,lat,lng
0,75001,Louvre,16252,48.864049,2.331053
1,75002,Bourse,20260,48.867564,2.343990
2,75003,Temple,34788,48.863480,2.359115
3,75004,Hôtel-de-Ville,27487,48.853428,2.358279
4,75005,Panthéon,59108,48.843491,2.351834
5,75006,Luxembourg,40916,48.848858,2.335422
6,75007,Palais-Bourbon,52512,48.854335,2.313403
7,75008,Élysée,36453,48.871872,2.317643
8,75009,Opéra,59629,48.879018,2.337906
9,75010,Entrepôt,91932,48.878562,2.360369


# Mapping Paris

In [43]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   4.94 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  16.46 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  19.11 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  21.15 MB/s


In [45]:
import folium # map rendering library

In [46]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [48]:
address = '75001, France'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of 75001, France are 48.86191335, 2.34683143191878.


In [56]:
map_paris = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for index,row in df_paris.iterrows():
    district,name,population,lat, lng = row.values
    label = '{} ({}), pop:{}'.format(name, district, population)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
map_paris

# Getting Foursquare Data

In [63]:
# The code was removed by Watson Studio for sharing.

In [59]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [61]:
def getNearbyVenues(df):
    
    radius=500
    venues_list=[]
    for (index,(district,name,population,lat, lng)) in df.iterrows():
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            district, 
            name,
            population,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                             'Name',
                             'Population',
                              'Latitude', 
                              ' Longitude', 
                              'Venue', 
                              'Venue Latitude', 
                              'Venue Longitude', 
                              'Venue Category']
    
    return(nearby_venues)

In [64]:
paris_venues = getNearbyVenues(df_paris)

Louvre
Bourse
Temple
Hôtel-de-Ville
Panthéon
Luxembourg
Palais-Bourbon
Élysée
Opéra
Entrepôt
Popincourt
Reuilly
Gobelins
Observatoire
Vaugirard
Passy
Batignolles-Monceau
Buttes-Montmartre
Buttes-Chaumont
Ménilmontant


In [65]:
paris_venues.head()

,District,Name,Population,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,75001,Louvre,16252,48.864049,2.331053,Les Arts Décoratifs,48.863077,2.333393,Art Museum
1,75001,Louvre,16252,48.864049,2.331053,Grand Bassin Rond,48.862880,2.329295,Fountain
2,75001,Louvre,16252,48.864049,2.331053,Librairie Galignani,48.864989,2.328570,Bookstore
3,75001,Louvre,16252,48.864049,2.331053,Hôtel Le Pradey,48.864459,2.331654,Hotel
4,75001,Louvre,16252,48.864049,2.331053,Balagan,48.865432,2.329680,Israeli Restaurant


In [67]:
paris_venues.shape

(1261, 9)

In [70]:
paris_venues.to_csv('paris_venues.csv',';')